In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/HW5

/content/gdrive/MyDrive/Colab Notebooks/HW5


In [3]:
# !pip install numpy==1.16.2
import numpy as np
print(np.__version__)
# !pip install torch
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
!pip install torchsummary 
from torchsummary import summary
from torch.autograd import Variable
!pip install pytorch_ssim
import pytorch_ssim

import math
from math import exp
import time
import matplotlib.pyplot as plt
import csv
import os
from os import path
import gc
import glob
from PIL import Image

1.19.5
  Created wheel for pytorch-ssim: filename=pytorch_ssim-0.1-py3-none-any.whl size=2026 sha256=7357c78b57e7a684402df4262c64dbed452df2446773f11037b461e220dc70ed
  Stored in directory: /root/.cache/pip/wheels/ee/20/09/ebf5e58bdf2560c760074cd140b7f7b0c882e216feabf1ae30
Successfully built pytorch-ssim


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18370058781790648910, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14512029696
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13301782278046349956
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [5]:
cuda = torch.cuda.is_available()
print(cuda)
num_workers = 2 if cuda else 0

True


In [6]:
transform = transforms.Compose([          
            transforms.ToTensor(),
            ])

In [34]:
class MyDataset(Dataset):
  def __init__(self, image_in_folder, image_out_folder=None, transform=None):
    self.image_in_folder = image_in_folder
    self.image_in_paths = sorted(os.listdir(image_in_folder))
    if image_out_folder is not None:
      self.image_out_folder = image_out_folder
      self.image_out_paths = sorted(os.listdir(image_out_folder))
    else:
      self.image_out_folder = None  
    self.transform = transform
            
  def __getitem__(self, index):
    x = Image.open(self.image_in_folder + self.image_in_paths[index])
    x = x.convert('RGB')
    if self.image_out_folder is not None:
      y = Image.open(self.image_out_folder + self.image_out_paths[index])
      y = y.convert('RGB')
    else:
      y = self.image_in_paths[index]
    if self.transform is not None:
        x = self.transform(x)
        p = transforms.Compose([transforms.Resize((x.size()[-2]*4, x.size()[-1]*4))])
        x = p(x)
        if self.image_out_folder is not None:
          y = self.transform(y)
    return x, y

  def __len__(self):
    return len(self.image_in_paths)

In [53]:
train_x = 'HW5_IDLS21/HW5_train_val/DIV2K_train_LR_bicubic/X4/'
train_y = 'HW5_IDLS21/HW5_train_val/DIV2K_train_HR/'
train_dataset = MyDataset(train_x, train_y, transform)
train_loader_args = dict(shuffle=True, batch_size=1, num_workers=num_workers, pin_memory=True, drop_last=True) \
                    if cuda else dict(shuffle=True, batch_size=1)
train_loader = DataLoader(train_dataset, **train_loader_args)
del train_x, train_y

val_x = 'HW5_IDLS21/HW5_train_val/DIV2K_valid_LR_bicubic/X4/'
val_y = 'HW5_IDLS21/HW5_train_val/DIV2K_valid_HR/'
val_dataset = MyDataset(val_x, val_y, transform)
val_loader_args = dict(shuffle=False, batch_size=1, num_workers=num_workers, pin_memory=True) \
                    if cuda else dict(shuffle=False, batch_size=1)
val_loader = DataLoader(val_dataset, **val_loader_args)
del val_x, val_y

# test_x_large = 'HW5_IDLS21/HW5_test/large_test/'
test_x_large = 'HW5_IDLS21/HW5_train_val/DIV2K_valid_LR_bicubic/X4/'
test_dataset = MyDataset(test_x_large, None, transform)
test_loader_args = dict(shuffle=False, batch_size=1, num_workers=num_workers, pin_memory=True) \
                    if cuda else dict(shuffle=False, batch_size=1)
test_loader = DataLoader(test_dataset, **test_loader_args)
del test_x_large

In [54]:
print(test_dataset.__getitem__(1)[0].size(), test_dataset.__getitem__(1)[1])

torch.Size([3, 1356, 2040]) 0802x4.png


In [55]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.l1 = nn.Conv2d(3, 64, 3, stride=1, padding=1, bias=False)
    self.l2 = nn.Conv2d(64, 64, 3, stride=1, padding=1, bias=False)
    self.l3 = nn.MaxPool2d(2, return_indices=True)
    self.d = nn.Dropout(0.1)
    self.l4 = nn.Conv2d(64, 128, 3, stride=1, padding=1, bias=False)
    self.l5 = nn.Conv2d(128, 128, 3, stride=1, padding=1, bias=False)
    self.l6 = nn.MaxPool2d(2, return_indices=True)
    self.l7 = nn.Conv2d(128, 256, 3, stride=1, padding=1, bias=False)
    
    self.l8 = nn.Conv2d(256, 128, 3, stride=1, padding=1, bias=False)
    self.l9 = nn.MaxUnpool2d(2)
    self.l10 = nn.Conv2d(128, 128, 3, stride=1, padding=1, bias=False)
    self.l11 = nn.Conv2d(128, 64, 3, stride=1, padding=1, bias=False)
    self.l12 = nn.MaxUnpool2d(2)
    self.l13 = nn.Conv2d(64, 64, 3, stride=1, padding=1, bias=False)
    self.l14 = nn.Conv2d(64, 3, 3, stride=1, padding=1, bias=False)

  def forward(self, x):
    y1 = self.l1(x)
    y1 = self.l2(y1)
    y2, i1 = self.l3(y1)
    y2 = self.d(y2)
    y2 = self.l4(y2)
    y2 = self.l5(y2)
    y3, i2 = self.l6(y2)
    y3 = self.l7(y3)
    y3 = self.l8(y3)
    y3 = self.l9(y3, i2)
    y3 = self.l10(y3)
    y3 = self.l11(y3)
    y3 = self.l12(y3, i1)
    y3 = self.l13(y3)
    y3 = self.l14(y3)
    # y3 = torch.clamp(y3, 0, 1)
    
    return y3

In [56]:
model = Model()
print(model)
model.cuda()
summary(model.cuda(), (3, 1024, 1024))
# summary(model.cuda(), (3, 1024, 1024), batch_size=-1, device='cuda')

Model(
  (l1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d): Dropout(p=0.1, inplace=False)
  (l4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l8): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l9): MaxUnpool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  (l10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l11): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l12): MaxUnpool2d

In [57]:
 
# Calculate the one-dimensional Gaussian distribution vector
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()
 
 
 # Create a Gaussian kernel, obtained by matrix multiplication of two one-dimensional Gaussian distribution vectors
 # You can set the channel parameter to expand to 3 channels
def create_window(window_size, channel=1):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window
 
 
 # Calculate SSIM
 # Use the formula of SSIM directly, but when calculating the average value, instead of directly calculating the pixel average value, normalized Gaussian kernel convolution is used instead.
 # The formula Var(X)=E[X^2]-E[X]^2, cov(X,Y)=E[XY]-E[X]E[Y] is used when calculating variance and covariance .
 # As mentioned earlier, the above expectation operation is replaced by Gaussian kernel convolution.
def ssim(img1, img2, window_size=11, window=None, size_average=True, full=False, val_range=None):
    # Value range can be different from 255. Other common ranges are 1 (sigmoid) and 2 (tanh).
    if val_range is None:
        if torch.max(img1) > 128:
            max_val = 255
        else:
            max_val = 1
 
        if torch.min(img1) < -0.5:
            min_val = -1
        else:
            min_val = 0
        L = max_val - min_val
    else:
        L = val_range
 
    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)
 
    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)
 
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
 
    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd, groups=channel) - mu1_mu2
 
    C1 = (0.01 * L) ** 2
    C2 = (0.03 * L) ** 2
 
    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = torch.mean(v1 / v2)  # contrast sensitivity
 
    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)
 
    if size_average:
        ret = ssim_map.mean()
    else:
        ret = ssim_map.mean(1).mean(1).mean(1)
 
    if full:
        return ret, cs
    return ret
 
 
 
# Classes to re-use window
class SSIM(torch.nn.Module):
    def __init__(self, window_size=11, size_average=True, val_range=None):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.val_range = val_range
 
        # Assume 1 channel for SSIM
        self.channel = 1
        self.window = create_window(window_size)
 
    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()
 
        if channel == self.channel and self.window.dtype == img1.dtype:
            window = self.window
        else:
            window = create_window(self.window_size, channel).to(img1.device).type(img1.dtype)
            self.window = window
            self.channel = channel
 
        return ssim(img1, img2, window=window, window_size=self.window_size, size_average=self.size_average)

In [58]:
def mypsnr(img1, img2):
    max_pixel = 1.0
    mse = np.mean((img1 - img2)**2)
    psnr = 20 * math.log(max_pixel / math.sqrt(mse), 10)
    
    return psnr
    
def myssim(img1, img2):
    #convert to tensor
    img1 = torch.tensor(img1).cuda()
    img2 = torch.tensor(img2).cuda()

    #convert to variable
    img1 = Variable(img1,  requires_grad=False)
    img2 = Variable(img2, requires_grad = True)
    
    # ssim_value = pytorch_ssim.ssim(img1, img2)
    ssim_value = ssim(img1, img2)
    
    return ssim_value.item()

In [59]:
def train_epoch(model, train_loader, criterion, optimiser):
    print("Training...")
    model.train()
    
    running_loss = 0.0

    start_time = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        optimiser.zero_grad()
        data = data.to(device)
        target = target.to(device)
        
        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()
        
        loss.backward()
        optimiser.step()
        
        if batch_idx % 100 == 0:
          print("Training batch: ", batch_idx)
            
    end_time = time.time()

    running_loss /= len(train_loader)
    print("Training loss: ", running_loss, "Time: ", end_time-start_time, "s")
        
    return running_loss

In [60]:
def val_epoch(model, val_loader, criterion):
    print("Validating...")
    with torch.no_grad():
      model.eval()
      
      running_loss = 0.0
      running_psnr = 0.0
      running_ssim = 0.0

      start_time = time.time()
      
      for batch_idx, (data, target) in enumerate(val_loader):
          data = data.to(device)
          target = target.to(device)
          
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()
      
          t,o = target.cpu().numpy(), outputs.cpu().numpy()
          running_psnr += mypsnr(t, o)
          running_ssim += myssim(t, o)
          
          if batch_idx % 50 == 0:
            print("Validation batch: ", batch_idx)
            
    end_time = time.time()
    
    running_loss /= len(val_loader)
    print("Validation loss: ", running_loss, "Time: ", end_time-start_time, "s")
    running_psnr/=len(val_loader)
    running_ssim/=len(val_loader)
    print("PSNR: ", running_psnr, "\tSSIM: ", running_ssim)
        
    return running_loss, running_psnr, running_ssim

In [61]:
n_epochs = 200
model_no = 5

learningRate = 0.00005
weightDecay = 1e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model()
model = model.to(device)

def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform(m.weight)
        # m.bias.data.fill_(0.01)

model.apply(init_weights)

criterion = nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr = learningRate, weight_decay= weightDecay)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode='min', factor=0.7, patience=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [ ]:
Train_loss = []
Val_loss = []
PSNR = []
SSIM = []

for i in range(n_epochs):
    print("Epoch ", i)

    if path.exists('model_'+ str(model_no) + '_' + str(i)):
      checkpoint = torch.load('model_' + str(model_no) + '_' + str(i))
      model.load_state_dict(checkpoint['model_state_dict'])
      # optimiser.load_state_dict(checkpoint['optimiser_state_dict'])
  
      Train_loss = checkpoint['tl']
      Val_loss = checkpoint['vl']
      PSNR = checkpoint['psnr']
      SSIM = checkpoint['ssim']
      
      continue
    print("LR: ", optimiser.param_groups[0]['lr'])
    
    train_loss = train_epoch(model, train_loader, criterion, optimiser)
    val_loss, p, s = val_epoch(model, val_loader, criterion)
    
    Train_loss.append(train_loss)
    Val_loss.append(val_loss)
    PSNR.append(p)
    SSIM.append(s)

    torch.save({
            'epoch': i,
            'model_state_dict': model.state_dict(),
            'optimiser_state_dict': optimiser.state_dict(),
            'tl': Train_loss,
            'vl': Val_loss,
            'psnr': PSNR,
            'ssim': SSIM
            }, 'model_' + str(model_no) + '_' + str(i))

Epoch  0
Epoch  1
Epoch  2
Epoch  3
Epoch  4
Epoch  5
Epoch  6
Epoch  7
Epoch  8
Epoch  9
Epoch  10
Epoch  11
Epoch  12
Epoch  13
Epoch  14
Epoch  15
Epoch  16
Epoch  17
Epoch  18
Epoch  19
Epoch  20
Epoch  21
Epoch  22
Epoch  23
Epoch  24
Epoch  25
Epoch  26
Epoch  27
Epoch  28
Epoch  29
Epoch  30
Epoch  31
Epoch  32
Epoch  33
Epoch  34
Epoch  35
Epoch  36
Epoch  37
Epoch  38
Epoch  39
Epoch  40
Epoch  41
Epoch  42
Epoch  43
Epoch  44
Epoch  45
Epoch  46
Epoch  47
Epoch  48
Epoch  49
Epoch  50
Epoch  51
Epoch  52
LR:  5e-05
Training...
Training batch:  0
Training batch:  100
Training batch:  200
Training batch:  300
Training batch:  400
Training batch:  500
Training batch:  600
Training batch:  700
Training loss:  0.004087499698925967 Time:  388.79089188575745 s
Validating...
Validation batch:  0
Validation batch:  50
Validation loss:  0.003886408029939048 Time:  53.805938720703125 s
PSNR:  25.113558812148348 	SSIM:  0.6559211325645447
Epoch  53
LR:  5e-05
Training...
Training batch: 

In [44]:
checkpoint = torch.load('AE/model_5_155')
model.load_state_dict(checkpoint['model_state_dict'])
# optimiser.load_state_dict(checkpoint['optimiser_state_dict'])

PSNR = checkpoint['psnr']
SSIM = checkpoint['ssim']

print(PSNR)  
print(SSIM)
print(np.argmax(PSNR), np.max(PSNR))
print(np.argmax(SSIM), np.max(SSIM))

for i in range(156):
  if PSNR[i]>26:
    print(i, PSNR[i], SSIM[i])

[-30.565415139425905, -26.46253641369647, -25.0797845124138, -24.120787228766893, -23.494197781299384, -22.393701837568198, -21.8003914346464, -19.917325460832807, -19.077667709215667, -17.69594463443075, -16.613640024000368, -15.957452968830554, -14.538428946515928, -13.806551430488014, -11.831229940827606, -10.39625203105006, -9.570058267771735, -5.9573417014697005, -2.9524351712121844, -1.5997595500282438, 1.5544628707788575, 3.2336339643333023, 4.922935554255063, 6.635804279485393, 8.493806856320104, 8.998151766877411, 4.599686928148439, 5.300740935173167, 5.723134432029325, 8.619687220558417, 9.678872803296635, 10.954322156995724, 10.940693434507894, 13.695514276248865, 14.679284278353043, 15.155377788875075, 15.925493951931548, 14.547681201300684, 18.503233472253385, 19.184767731754846, 20.089535219720002, 20.781206351757838, 21.876316268126796, 22.545528923684834, 23.871586499318553, 24.26445822902237, 24.725813276387026, 23.444321541321713, 24.535938077011437, 24.41838770348258

In [63]:
def test_epoch(model, test_loader):
    print("Testing...")
    model.eval()
    start_time = time.time()

    with torch.no_grad():
      for batch_idx, (data, label) in enumerate(test_loader):
        data = data.cuda()
        batch_size = data.size(0)
        fake_img = model(data)
        # np.save("Test_5_111/" + label[0][:-4], fake_img[0].cpu().numpy().transpose(1,2,0))
        save_image(fake_img[0].cpu(), "Test_AE_5_111_Valid/" + label[0])
        del fake_img
        torch.cuda.empty_cache()
    
    end_time = time.time()

In [64]:
checkpoint = torch.load('AE/model_5_111')
model.load_state_dict(checkpoint['model_state_dict'])
!mkdir Test_AE_5_111_Valid
test_epoch(model, test_loader)

mkdir: cannot create directory ‘Test_AE_5_111_Valid’: File exists
Testing...
